In [14]:
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig, ComputeTarget
from azureml.core.runconfig import DockerConfiguration

In [4]:
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [7]:
exp = Experiment(ws, 'test_running_p2mpp_pytorch')

In [11]:
target = ComputeTarget(ws, "OMENE-A100VM")

In [30]:
docker_config = DockerConfiguration(use_docker=True)

myenv = Environment.from_pip_requirements("p2mpp_pytorch_env", "./requirements.txt")
myenv.docker.base_image = None
myenv.docker.base_dockerfile = "./Dockerfile"
myenv.python.user_managed_dependencies = True

In [31]:
myenv.register(workspace=ws)

{
    "assetId": "azureml://locations/southcentralus/workspaces/839c777b-7627-4c33-9a14-9516a8ae2a93/environments/p2mpp_pytorch_env/versions/2",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "FROM mcr.microsoft.com/aifx/acpt/stable-ubuntu2004-cu117-py38-torch201:biweekly.202310.3\n\nRUN rm -rf /usr/lib/x86_64-linux-gnu/libnvidia-ml.so.1 /usr/lib/x86_64-linux-gnu/libcuda.so.1 /usr/lib/x86_64-linux-gnu/libcudadebugger.so.1\n\n\n# Install pip dependencies\nCOPY requirements.txt .\nRUN pip install -r requirements.txt --no-cache-dir\n\nCOPY external/chamfer ./external/chamfer\nCOPY external/neural_renderer ./external/neural_renderer\n\nRUN cd external/chamfer && python setup.py install\nRUN cd external/neural_renderer && python setup.py install\n\n# Inference requirements\nCOPY --from=mcr.microsoft.com

In [32]:
myenv.build(ws)

Building a non-registered environment is not supported. Registering environment.


In [ ]:
config = ScriptRunConfig(
    source_directory='.',
    script='train_mvp2m.py',
    compute_target=target,
    environment=myenv,
    arguments = [
        'fit', "--config", "configs/default.yaml"
    ],
    docker_runtime_config=docker_config
)
